In [5]:
import os
import sys
print(os.getcwd())
# change working directory to project root
import platform
if platform.system() is "Windows":
    os.chdir('E:\\sven\\git\\cardio')
else:
    os.chdir('/Users/minority/Code/Git/cardio')
print(os.getcwd())
from src.utils.utils_io import Console_and_file_logger
from src.utils.file_wrapper.app.Files import Basefile, Dicomfile
from src.utils.file_wrapper.app.IO import FileWalker
from src.utils.myshow import myshow, myshow3d
# enable inline plotting

import matplotlib.pyplot as plt
%matplotlib inline
from ipywidgets import interact, fixed

import logging
Console_and_file_logger('series_Reader_1', logging.INFO)

import re
import numpy as np
from scipy import stats
from collections import OrderedDict
import json
import SimpleITK as sitk

E:\sven\git\cardio
E:\sven\git\cardio


In [8]:
# reads a dicom serie reshape it at 

series_path = '../../ahf_export/0059-2WHFP73X-2007-03-28/'
out_file = './data/processed/series_reader/series_reader.nrrd'

#1.3.46.670589.11.0.0.11.4.2.0.15002.5.808.2006062910232843457


import SimpleITK as sitk
import sys, os

if len ( series_path ) < 3:
    print( "Usage: DicomSeriesReader <input_directory> <output_file>" )
    sys.exit ( 1 )


print( "Reading Dicom directory:", series_path )
reader = sitk.ImageSeriesReader()


dicom_names = reader.GetGDCMSeriesFileNames( series_path )
print('# dicom: {}'.format(len(dicom_names)))
reader.SetFileNames(dicom_names)
reader.MetaDataDictionaryArrayUpdateOn()
reader.LoadPrivateTagsOn()
image = reader.Execute()

Reading Dicom directory: ../../ahf_export/0059-2WHFP73X-2007-03-28/
# dicom: 0


RuntimeError: Exception thrown in SimpleITK ImageSeriesReader_Execute: C:\Users\dashboard\Miniconda3\conda-bld\simpleitk_1546529802744\work\Code\IO\src\sitkImageSeriesReader.cxx:130:
sitk::ERROR: File names information is empty. Cannot read series.

In [52]:
size = image.GetSize()
dimension = image.GetDimension()

#256 256 10 25
print( "Image size:", size[0], size[1], size[2] )
print( "Image dimension: {}".format(dimension ))
img_nda = sitk.GetArrayFromImage(image)
print('shape of array {} '.format(img_nda.shape))


new_img_nda = np.reshape(img_nda, (25, 10, 256, 256))
new_img_nda.astype(np.float32)
print('shape of new array: {}'.format(new_img_nda.shape))

# dont work
# new_img = sitk.GetImageFromArray(new_img_nda, isVector=True)
# print(new_img_nda.dtype)


tdim = new_img_nda.shape[0]
slices = []
for t in range(tdim):
    slice = sitk.GetImageFromArray( new_img_nda[t], False )
    print(slice.GetSize())
    slices.append(slice)
    
#myshow3d(slices[1])

new_img = sitk.JoinSeries(slices)

# copy image information
for tag in slices[0].GetMetaDataKeys():
    try:
        value = slices[0].GetMetaData(tag)
        new_img.SetMetadata(tag, value)
    except:
        pass


size = new_img.GetSize()
dimension = new_img.GetDimension()
#256 256 10 25
print( "Image size: {}".format(size) )
print( "Image dimension: {}".format(dimension ))
print( "Writing image:", outout_file )

sitk.WriteImage( new_img, out_file )

#if ( not "SITK_NOSHOW" in os.environ ):
#    sitk.Show( image, "Dicom Series" )

Reading Dicom directory: ./data/raw/0007-0HQQW4ZN-2007-05-23/1.3.46.670589.11.0.0.11.4.2.0.5070.5.5892.2007052314282550951
# dicom: 250
Image size: 256 256 250
Image dimension: 3
shape of array (250, 256, 256) 
shape of new array: (25, 10, 256, 256)
(256, 256, 10)
(256, 256, 10)
(256, 256, 10)
(256, 256, 10)
(256, 256, 10)
(256, 256, 10)
(256, 256, 10)
(256, 256, 10)
(256, 256, 10)
(256, 256, 10)
(256, 256, 10)
(256, 256, 10)
(256, 256, 10)
(256, 256, 10)
(256, 256, 10)
(256, 256, 10)
(256, 256, 10)
(256, 256, 10)
(256, 256, 10)
(256, 256, 10)
(256, 256, 10)
(256, 256, 10)
(256, 256, 10)
(256, 256, 10)
(256, 256, 10)
Image size: (256, 256, 10, 25)
Image dimension: 4
Writing image: ./data/processed/series_reader/series_reader.nrrd


In [11]:
#
#  This example shows how to read a specific series from a Dicom directory that
#  may contain more than one series.  The script scans for all series.  If an
#  output name is given, it writes out the requested series.  If no specific
#  series name is given, the first series found is written.
#

from __future__ import print_function

import sys, getopt
import SimpleITK as sitk

target_series = ""
output_image = ""

target_series = '1.3.46.670589.11.0.0.11.4.2.0.15002.5.808.2006062910232843457'
input_directory = '/Users/minority/Code/Git/cardio/data/raw/ahf_export/0019-1EMMCVKN-2006-06-29/1.3.46.670589.11.0.0.11.4.2.0.15002.5.808.2006062910232843457'

output_image = '/Users/minority/Code/Git/cardio/data/processed/series_test/image.nrrd'

# Find the Dicom series
reader = sitk.ImageSeriesReader()
written = False

series_found = reader.GetGDCMSeriesIDs(input_directory)
print(len(series_found))

# Process each Dicom series
if len(series_found):

    for serie in series_found:

        print( "\nSeries:", serie )

        # Get the Dicom filename corresponding to the current series
        dicom_names = reader.GetGDCMSeriesFileNames(input_directory, serie)

        print( "\nFiles in series: ", dicom_names )

        if len(dicom_names):
            reader.SetFileNames(dicom_names)
            image = reader.Execute()
            print( "\nImage size: ", image.GetSize() )

            if (output_image != "") and not written:
                if (target_series == "" or target_series == serie):

                    print( "\nWriting", output_image )
                    sitk.WriteImage(image, output_image)
                    written = True


print ()

1

Series: 1.3.46.670589.11.0.0.11.4.2.0.15002.5.808.2006062910232843457

Files in series:  ('/Users/minority/Code/Git/cardio/data/raw/ahf_export/0019-1EMMCVKN-2006-06-29/1.3.46.670589.11.0.0.11.4.2.0.15002.5.808.2006062910232843457/1.3.12.2.1107.5.8.3.485257.836650.79844849.2006062911215356.dcm', '/Users/minority/Code/Git/cardio/data/raw/ahf_export/0019-1EMMCVKN-2006-06-29/1.3.46.670589.11.0.0.11.4.2.0.15002.5.808.2006062910232843457/1.3.12.2.1107.5.8.3.485257.836650.79844849.2006062911215471.dcm', '/Users/minority/Code/Git/cardio/data/raw/ahf_export/0019-1EMMCVKN-2006-06-29/1.3.46.670589.11.0.0.11.4.2.0.15002.5.808.2006062910232843457/1.3.12.2.1107.5.8.3.485257.836650.79844849.2006062911215567.dcm', '/Users/minority/Code/Git/cardio/data/raw/ahf_export/0019-1EMMCVKN-2006-06-29/1.3.46.670589.11.0.0.11.4.2.0.15002.5.808.2006062910232843457/1.3.12.2.1107.5.8.3.485257.836650.79844849.2006062911215677.dcm', '/Users/minority/Code/Git/cardio/data/raw/ahf_export/0019-1EMMCVKN-2006-06-29/1.3.4

In [9]:
# getImageFromArray doesn't work for 4D

nda = np.arange(210, dtype=np.float32).reshape([3,5,7,2])
img = sitk.GetImageFromArray(nda, isVector=True)
print(img.GetSize())

(7, 5, 3)


In [50]:
# print metadata of a single dicom file

from __future__ import print_function

import SimpleITK as sitk
import sys, os

reader = sitk.ImageFileReader()
reader.SetFileName( './data/processed/series_reader/series_reader.nrrd' )
reader.LoadPrivateTagsOn();
reader.ReadImageInformation();
reader.Execute()

for k in reader.GetMetaDataKeys():
    v = reader.GetMetaData(k)
    print("({0}) = = \"{1}\"".format(k,v))


(ITK_InputFilterName) = = "NrrdImageIO"
(NRRD_kinds[0]) = = "domain"
(NRRD_kinds[1]) = = "domain"
(NRRD_kinds[2]) = = "domain"
(NRRD_kinds[3]) = = "domain"


In [49]:
# works only if all series are in one folder

data_directory = './data/processed/0706_all_in_one_folder/'


reader = sitk.ImageSeriesReader()
seriesIDs = reader.GetGDCMSeriesIDs(data_directory)
reader.
print(set(seriesIDs))
#dicom_names = reader.GetGDCMSeriesFileNames( data_directory )
#print(dicom_names)

series_file_names = reader.GetGDCMSeriesFileNames(data_directory, '1.2.826.0.1.3680043.2.1125.1.10080851189844906763606927700990560')
print(series_file_names)
#reader.SetFileNames(series_file_names)
#img = reader.Execute()
#print(img.GetSize())

#print(seriesIDs)

{'1.2.826.0.1.3680043.2.1125.1.29412286761855053085686461185823830', '1.2.826.0.1.3680043.2.1125.1.64487671323249317086876887381820391', '1.2.826.0.1.3680043.2.1125.1.17941176333250487340853609277811877', '1.2.826.0.1.3680043.2.1125.1.32794806591937863613506325966455130', '1.2.826.0.1.3680043.2.1125.1.54111894849809494011931737595536716', '1.2.826.0.1.3680043.2.1125.1.6035568043445202033379912525043594', '1.2.826.0.1.3680043.2.1125.1.6068548146651530087021040043240670', '1.2.826.0.1.3680043.2.1125.1.65428943367291732170670687717276152', '1.2.826.0.1.3680043.2.1125.1.62958164148167491403616776061203050', '1.2.826.0.1.3680043.2.1125.1.61864610348359966091136322910448577', '1.2.826.0.1.3680043.2.1125.1.24229832886716854199904896826391891', '1.2.826.0.1.3680043.2.1125.1.86061619364594331591858186908531733', '1.2.826.0.1.3680043.2.1125.1.29013647632765653265820431668038768', '1.2.826.0.1.3680043.2.1125.1.86201943446447629773453589248111548', '1.2.826.0.1.3680043.2.1125.1.2896652938629371829

In [41]:
filtered_ids = ['1.2.826.0.1.3680043.2.1125.1.10080851189844906763606927700990560', '1.2.826.0.1.3680043.2.1125.1.10271684598644160001627510131546437']